In [33]:
import pandas as pd
import geocoder
import json

In [31]:
spb_clear_data_path = "C:/Users/user/Documents/GitHub/SpbZeroVision/Data/SpbData/ClearData/"
spb_processing_data_path = "C:/Users/user/Documents/GitHub/SpbZeroVision/Data/SpbData/ProcessingData/"

In [6]:
dtp_info_df = pd.read_csv(spb_clear_data_path + "spb_main_info.csv")

In [14]:
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display
    
    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = size / 200     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)
    
    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

In [30]:
geocoded_cards = []
for card in log_progress(dtp_info_df.iterrows(),10):
    row = card[1]
    r = geocoder.yandex(str(row.n_p) + " " + str(row.street) + " " + str(row.house))
    
    geocoded_card = r.json
    
    if(geocoded_card):
        geocoded_card['KartId'] = row['KartId']
    else:
        geocoded_card = {}
    
    geocoded_cards.append(geocoded_card)

Status code Unknown from https://geocode-maps.yandex.ru/1.x/: ERROR - HTTPSConnectionPool(host='geocode-maps.yandex.ru', port=443): Max retries exceeded with url: /1.x/?geocode=%D0%B3+%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3+%D1%83%D0%BB+%D0%A1%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0+28&lang=en-US&kind=&format=json&results=1 (Caused by ConnectTimeoutError(<urllib3.connection.VerifiedHTTPSConnection object at 0x000001A2D2237E80>, 'Connection to geocode-maps.yandex.ru timed out. (connect timeout=5.0)'))
Status code Unknown from https://geocode-maps.yandex.ru/1.x/: ERROR - HTTPSConnectionPool(host='geocode-maps.yandex.ru', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://geocode-maps.yandex.ru/1.x/: ERROR - HTTPSConnectionPool(host='geocode-maps.yandex.ru', port=443): Read timed out. (read timeout=5.0)
Status code Unknown from https://geocode-maps.yandex.ru/1.x/: ERROR - HTTPSConnectionPool(host='geocode-maps.yandex.ru', por

In [34]:
with open(spb_processing_data_path + "geocod_dtp_cards.json", 'w', encoding='utf-8') as out_file:
    json.dump(geocoded_cards, out_file)

In [41]:
#check original coords and geocoded coogez
geocoded_cards_simple = [
    {
        "KartId": g['KartId'] if 'KartId' in g else 0,
        "accuracy": g['accuracy'],
        "lat": g['lat'],
        "lng": g['lng'],
    }
for g in geocoded_cards]

geocoded_coords_df = pd.DataFrame(geocoded_cards)
geocoded_coords_df.to_csv(spb_processing_data_path + "geocoeded_dtp_coords.csv", encoding='utf-8')

KeyError: 'accuracy'

In [39]:
geocoded_cards[0]

{'accuracy': 'exact',
 'address': 'Russia, Saint Petersburg, Liteyny prospekt, 51',
 'country': 'Russia',
 'country_code': 'RU',
 'description': 'Saint Petersburg, Russia',
 'lat': '59.937197',
 'lng': '30.347776',
 'ok': True,
 'quality': 'house',
 'raw': {'metaDataProperty': {'GeocoderMetaData': {'kind': 'house',
    'text': 'Russia, Saint Petersburg, Liteyny prospekt, 51',
    'precision': 'exact',
    'Address': {'country_code': 'RU',
     'postal_code': '191014',
     'formatted': 'Saint Petersburg, Liteyny prospekt, 51',
     'Components': [{'kind': 'country', 'name': 'Russia'},
      {'kind': 'province', 'name': 'Severo-Zapadny federalny okrug'},
      {'kind': 'province', 'name': 'Saint Petersburg'},
      {'kind': 'locality', 'name': 'Saint Petersburg'},
      {'kind': 'street', 'name': 'Liteyny prospekt'},
      {'kind': 'house', 'name': '51'}]},
    'AddressDetails': {'Country': {'AddressLine': 'Saint Petersburg, Liteyny prospekt, 51',
      'CountryNameCode': 'RU',
      'C